# Experiment 1

### Environment Setup

In [1]:
%%bash -s

# set root folder and get environment
export root="/home/fabric/work"
source $root/env

In [2]:
# imports
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import uuid
from collections import defaultdict

# constants
root = "/home/fabric/work"

# component names
slice_name = "experiment1"
server_site, client_site = fablib.get_random_sites(count=2)

# switch detail
client_switch_name = "client-switch"
server_switch_name = "server-switch"
switch_cores = 2
switch_ram = 8
switch_disk = 100

# host details
client_host_name = "client-host"
server_host_name = "server-host"
image = 'default_ubuntu_20'
server_host_subnet=IPv4Network('10.0.1.0/24')
server_host_addr=IPv4Address('10.0.1.1')
client_host_subnet=IPv4Network('10.0.2.0/24')
client_host_addr=IPv4Address('10.0.2.2')

# network details
server_switch_net_name = "net-server-switch"
client_switch_net_name = "net-client-switch"
switch_switch_net_name = "net-switch-switch"

In [3]:
# some helper function
__id_counter = defaultdict(int)
def id(name=None):
  global __id_counter
  __id_counter[name] += 1
  return f"{name}-{str(__id_counter[name]).zfill(4)}"

## Setup Network

In [5]:
try:
  # create the slice
  slice = fablib.new_slice(name=slice_name)

  # add server switch
  server_switch = slice.add_node(name=server_switch_name, site=server_site, image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
  server_switch_iface_host = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
  server_switch_iface_client_switch = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

  # add client switch
  client_switch = slice.add_node(name=client_switch_name, site=client_site, image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
  client_switch_iface_host = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
  client_switch_iface_server_switch = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

  # add server host
  server_host = slice.add_node(name=server_host_name, site=server_site, image=image)
  server_host_iface = server_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

  # add client host
  client_host = slice.add_node(name=client_host_name, site=client_site, image=image)
  client_host_iface = client_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

  # add networks
  server_switch_net = slice.add_l2network(name=server_switch_net_name, interfaces=[server_switch_iface_host, server_host_iface])
  client_switch_net = slice.add_l2network(name=client_switch_net_name, interfaces=[client_switch_iface_host, client_host_iface])
  switch_switch_net = slice.add_l2network(name=switch_switch_net_name, interfaces=[server_switch_iface_client_switch, client_switch_iface_server_switch])

  slice.submit()
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()



-----------  ------------------------------------
Slice Name   experiment1
Slice ID     c00541eb-522b-4439-b30d-5926d023dc34
Slice State  Configuring
Lease End    2022-10-30 01:19:47 +0000
-----------  ------------------------------------

Retry: 48, Time: 608 sec

ID                                    Name           Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  -------------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
21e20526-9d12-4403-8435-b2fc7a9dc875  server-switch  DALL    dall-w2.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:400:a100:3000:f816:3eff:fece:29b0  Active
256a3729-c916-43b0-bdcf-6c5a2c239652  client-switch  TACC    tacc-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  129.114.110.123                         Active

Traceback (most recent call last):
  File "/tmp/ipykernel_208/3668902574.py", line 28, in <module>
    slice.submit()
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 1224, in submit
    self.wait_jupyter(timeout=wait_timeout, interval=wait_interval)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 1153, in wait_jupyter
    raise Exception(f"Timeout {timeout} sec exceeded in Jupyter wait")
Exception: Timeout 600 sec exceeded in Jupyter wait
